In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras import utils, Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
import matplotlib.pyplot as plt
data = pd.read_csv('./train.csv')
#print(data)

In [127]:
a = [1,2,3,43,5]
print(a[1:])

[2, 3, 43, 5]


In [4]:
train_image = data.iloc[:, 1:].to_numpy().astype('float32')/255
train_image = train_image.reshape(train_image.shape[0], 28, 28, 1).astype('float32')
train_label = data.loc[:, 'label'].to_numpy()
TrainOneHot = utils.to_categorical (train_label)

#print(train_label[:5])
#print(TrainOneHot[:5])
#print(train_image)
print("image.shape = ", train_image.shape, "\nlabel.shape = ", train_label.shape, "\nTrainOneHot.shape = ",TrainOneHot.shape)

image.shape =  (42000, 28, 28, 1) 
label.shape =  (42000,) 
TrainOneHot.shape =  (42000, 10)


In [5]:
#CNN model
model2 = Sequential  ()
model2.add ( Conv2D (16, kernel_size=(5,5), padding='same', input_shape = (28,28,1),
           activation='relu'))
model2.add (MaxPooling2D (pool_size=(2,2)))

model2.add ( Conv2D (32, kernel_size=(5,5), 
       padding='same', activation='relu' ) )
model2.add ( MaxPooling2D (pool_size=(2,2)))
model2.add ( Dropout (0.5) )
model2.add ( Flatten ( ) )

model2.add ( Dense (128, activation='relu' ) )
model2.add ( Dropout (0.5) )

model2.add ( Dense (10, activation='softmax' ) )

model2.compile ( loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model2.summary ( ) 

In [6]:
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
adam = optimizers.Adam(learning_rate = 0.001)
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 30)
mc = ModelCheckpoint('Model/CNN.h5', monitor = 'val_loss', mode = 'min', verbose = 1, save_best_only = True)
callbacks_list = [mc, es]

In [7]:
CNN_history=model2.fit (train_image, TrainOneHot, validation_split=0.2, epochs=10, batch_size=128)

Epoch 1/10
263/263 [==============================] - 8s 31ms/step - loss: 0.5312 - accuracy: 0.8296 - val_loss: 0.1078 - val_accuracy: 0.9668
Epoch 2/10
263/263 [==============================] - 8s 29ms/step - loss: 0.1727 - accuracy: 0.9479 - val_loss: 0.0747 - val_accuracy: 0.9768
Epoch 3/10
263/263 [==============================] - 8s 29ms/step - loss: 0.1211 - accuracy: 0.9618 - val_loss: 0.0536 - val_accuracy: 0.9818
Epoch 4/10
263/263 [==============================] - 8s 29ms/step - loss: 0.1001 - accuracy: 0.9690 - val_loss: 0.0467 - val_accuracy: 0.9843
Epoch 5/10
263/263 [==============================] - 8s 29ms/step - loss: 0.0893 - accuracy: 0.9738 - val_loss: 0.0440 - val_accuracy: 0.9861
Epoch 6/10
263/263 [==============================] - 8s 29ms/step - loss: 0.0808 - accuracy: 0.9753 - val_loss: 0.0426 - val_accuracy: 0.9854
Epoch 7/10
263/263 [==============================] - 8s 29ms/step - loss: 0.0709 - accuracy: 0.9776 - val_loss: 0.0403 - val_accuracy: 0.9873

In [8]:
data = pd.read_csv('./test.csv')
test_image = data.iloc[:, :].to_numpy().astype('float32')/255
test_image = test_image.reshape(test_image.shape[0], 28, 28, 1).astype('float32')


In [9]:
result = model2.predict_classes(test_image).flatten()
print(result.shape)


Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
(28000,)


In [10]:
import csv
with open('submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['ImageId', 'Label']
    csv_writer.writerow(header)
    for i in range(result.shape[0]):
        row = [i+1, result[i]]
        csv_writer.writerow(row)